# Kundur Two Areas

## Run simulation

### Import Libraries

In [ ]:
import sys, os

notebook_dir = os.path.abspath('')
dpsim_root_dir = os.path.join(notebook_dir, "../../..")

sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim'))
sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

import matpower
import dpsimpy

from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import urllib.request
import matplotlib.pyplot as plt

### Simulation parameters

In [ ]:
# simulation files
path_static_file = os.path.join(notebook_dir, 'Kundur2Areas/Kundur2Areas.mat')
path_dynamic_file = os.path.join(notebook_dir, 'Kundur2Areas/Kundur2Areas_dyn.mat')


### 1. Powerflow for initialization

In [ ]:
sim_name_pf = 'Kundur2Areas_PF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# read and create dpsim topology
mpc_reader = matpower.Reader(mpc_file_path=path_static_file, mpc_name='Kundur2Areas',
                             mpc_dyn_file_path=path_dynamic_file, mpc_dyn_name='Kundur2Areas_dyn')
mpc_reader.load_mpc(domain=matpower.Domain.PF)
system_pf = mpc_reader.system

# log results
logger = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)

# Parametrize and run simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.info)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(1)
sim_pf.set_final_time(0.1)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.add_logger(logger)
sim_pf.run()

### Read pf results

In [ ]:
dpsim_result_file = 'logs/' + sim_name_pf + '/' + sim_name_pf + '.csv'
ts_dpsim_pf = read_timeseries_csv(dpsim_result_file)

#### DPsim results

In [ ]:
dpsim_results = pd.DataFrame(columns=['Bus', 'V_mag(pu)', 'V_angle(rad)', 'P(pu)', 'Q(pu)'])
base_power = 100 #mw
for i in range(len(system_pf.nodes)):
    node_name = system_pf.nodes[i].name() #ex. N5
    node_number = node_name.replace('N', '')
    node_baseV = mpc_reader.mpc_bus_data.loc[mpc_reader.mpc_bus_data['bus_i'] == int(node_number), 'baseKV'].iloc[0] * 1e3
    w_mw = 1e-6
    dpsim_results.loc[i] = ([node_name] + [round(np.absolute(ts_dpsim_pf[node_name + '.V'].values[-1]) / node_baseV, 5)]
        + [round(np.angle(ts_dpsim_pf[node_name + '.V'].values[-1]), 5)] 
        + [round(w_mw * np.real(ts_dpsim_pf[node_name + '.S'].values[-1]) / base_power, 4)] 
        + [round(w_mw * np.imag(ts_dpsim_pf[node_name + '.S'].values[-1]) / base_power, 5)])

dpsim_results

### Validate Results against PSAT

In [ ]:
### PSAT Results
voltage_mag = np.array([1.03, 1.01, 1.03, 1.01, 1.0074, 0.98052, 0.9653, 0.95369, 0.97637, 0.98626, 1.0094])
voltage_angle = np.array([0.46853, 0.29839, 0, -0.17728, 0.35586, 0.18031, 0.03435, -0.20555, -0.44054, -0.29467, -0.11543])
p_loads = np.array([0, 0, 0, 0, 0, 0, 9.67, 0, 17.67, 0, 0])
q_loads = np.array([0, 0, 0, 0, 0, 0, -1, 0, -2.5, 0, 0])
p_gen = np.array([7, 7, 7.1849, 7, 0, 0, 0, 0, 0, 0, 0])
q_gen = np.array([1.7897, 2.2005, 1.6875, 1.8498, 0, 0, 0, 0, 0, 0, 0])
busses = ["N1", "N2", "N3", "N4", "N5", "N6", "N7", "N8", "N9", "N10", "N11"]
d = {"Bus": busses, "V_mag(pu)": voltage_mag, "V_angle(rad)": voltage_angle, "P(pu)": -p_loads+p_gen, "Q(pu)": -q_loads+q_gen}
psat_results = pd.DataFrame(d)

psat_results

### Difference

In [ ]:
difference_vmag = psat_results['V_mag(pu)'] - dpsim_results['V_mag(pu)']
difference_vmag[0:3] = difference_vmag[0:3] * 20000
difference_vmag[4:-1] = difference_vmag[4:-1] * 230000
difference_vangle = psat_results['V_angle(rad)'] - dpsim_results['V_angle(rad)']

#base power = 100MW
difference_p = (psat_results['P(pu)'] - dpsim_results['P(pu)'])*100*1e6
difference_q = (psat_results['Q(pu)'] - dpsim_results['Q(pu)'])*100*1e6
difference = {"Bus": busses, "V_mag[V]": difference_vmag, "V_angle [rad]": difference_vangle, "P[W]": difference_p, "Q[VAr]": difference_q}
difference = pd.DataFrame(difference)

difference

### Assertion

In [ ]:
print(np.max(np.absolute(difference_p)))
assert(np.all(np.absolute(difference_p)<1e-12))    #Watt
print(np.max(np.absolute(difference_q)))
assert(np.all(np.absolute(difference_q)<5000.1))    #Watt

print(np.max(difference_vmag))
assert(np.all(difference_vmag)<7)

print(np.max(difference_vangle))
assert(np.all(difference_vangle)<1e-12)

### 2. Dynamic Simulation at node 8

#### Declare some functions

In [ ]:
def Kundur2Areas_Fault(domain="EMT", reduce_inertia=False):

    if domain=="SP":
        matpower_domain = matpower.Domain.SP
        dpsim_domain = dpsimpy.Domain.SP
    elif domain=="DP":
        matpower_domain = matpower.Domain.DP
        dpsim_domain = dpsimpy.Domain.DP
    else:
        domain=="EMT"
        matpower_domain = matpower.Domain.EMT
        dpsim_domain = dpsimpy.Domain.EMT
        
    sim_name_fault = domain + "_Kundur2Areas_Fault"
    dpsimpy.Logger.set_log_dir('logs/' + sim_name_fault)

    mpc_reader2 = matpower.Reader(mpc_file_path=path_static_file, mpc_name='Kundur2Areas',
                                  mpc_dyn_file_path=path_dynamic_file, mpc_dyn_name='Kundur2Areas_dyn')
    mpc_reader2.create_dpsim_objects(domain=matpower_domain, frequency=60, 
                                     with_avr=False, with_tg=False, with_pss=False)

    ### Extend topology with switch between node N15 and groud
    sw=None
    switch_closed = 0.001 * (230*230/100)
    if domain=="SP":
        sw = dpsimpy.sp.ph1.Switch('Fault', dpsimpy.LogLevel.info)
        sw.set_parameters(1e18, switch_closed)
        sw.open()
        mpc_reader2.dpsimpy_comp_dict['Fault'] = [sw]
        mpc_reader2.dpsimpy_comp_dict['Fault'].append([dpsimpy.sp.SimNode.gnd, mpc_reader2.dpsimpy_busses_dict["N8"]])
    elif domain=="DP":
        sw = dpsimpy.dp.ph1.Switch('Fault', dpsimpy.LogLevel.info)
        sw.set_parameters(1e18, switch_closed)
        sw.open()
        mpc_reader2.dpsimpy_comp_dict['Fault'] = [sw]
        mpc_reader2.dpsimpy_comp_dict['Fault'].append([dpsimpy.dp.SimNode.gnd, mpc_reader2.dpsimpy_busses_dict["N8"]])
    else:
        sw = dpsimpy.emt.ph3.SeriesSwitch('Fault', dpsimpy.LogLevel.info)
        sw.set_parameters(1e18, switch_closed)
        sw.open()
        mpc_reader2.dpsimpy_comp_dict['Fault'] = [sw]
        mpc_reader2.dpsimpy_comp_dict['Fault'].append([dpsimpy.emt.SimNode.gnd, mpc_reader2.dpsimpy_busses_dict["N8"]])
    
    # scale inertia constant of geners
    if (reduce_inertia):
        mpc_reader2.dpsimpy_comp_dict['Gen_N1'][0].scale_inertia_constant(0.7)
        mpc_reader2.dpsimpy_comp_dict['Gen_N2'][0].scale_inertia_constant(0.7)
        mpc_reader2.dpsimpy_comp_dict['Gen_N3'][0].scale_inertia_constant(0.7)
        mpc_reader2.dpsimpy_comp_dict['Gen_N4'][0].scale_inertia_constant(0.7)

    # create dpsim topology
    mpc_reader2.create_dpsim_topology()

    #initialize node voltages using pf results
    system_dyn = mpc_reader2.system
    system_dyn.init_with_powerflow(system_pf, dpsim_domain)

    # log results
    logger = dpsimpy.Logger(sim_name_fault)
    for node in system_dyn.nodes:
        logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute("line3_7-8.I", 'i_intf', mpc_reader2.dpsimpy_comp_dict['line3_7-8'][0])
    logger.log_attribute("line4_7-8.I", 'i_intf', mpc_reader2.dpsimpy_comp_dict['line4_7-8'][0])
    logger.log_attribute("line5_8-9.I", 'i_intf', mpc_reader2.dpsimpy_comp_dict['line5_8-9'][0])
    logger.log_attribute("line6_8-9.I", 'i_intf', mpc_reader2.dpsimpy_comp_dict['line6_8-9'][0])
    logger.log_attribute("Switch.I", 'i_intf', sw)

    # Parametrize and run simulation
    sim = dpsimpy.Simulation(sim_name_fault, dpsimpy.LogLevel.info)
    sim.set_system(system_dyn)
    if domain=="EMT":
        sim.set_time_step(1e-4)
    else:
        sim.set_time_step(1e-4)
    sim.set_final_time(10)
    sim.set_domain(dpsim_domain)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.SparseLU)
    sim.do_init_from_nodes_and_terminals(True)
    sim.add_logger(logger)
    sim.do_system_matrix_recomputation(True)

    # add events
    sw_event_1 = dpsimpy.event.SwitchEvent(1, sw, True)
    sw_event_2 = dpsimpy.event.SwitchEvent(1.05, sw, False)
    sim.add_event(sw_event_1)
    sim.add_event(sw_event_2)

    sim.run()
    
    return sim_name_fault

#### SP Simulation

In [ ]:
sim_name_fault_sp = Kundur2Areas_Fault(domain="SP")
sim_name_fault_sp_inertia_reduction = Kundur2Areas_Fault(domain="SP", reduce_inertia=True)

#### DP Simulation

In [ ]:
sim_name_fault_dp = Kundur2Areas_Fault(domain="DP")
sim_name_fault_dp_inertia_reduction = Kundur2Areas_Fault(domain="DP", reduce_inertia=True)

#### EMT Simulation

In [ ]:
sim_name_fault_emt = Kundur2Areas_Fault(domain="EMT")
sim_name_fault_emt_inertia_reduction = Kundur2Areas_Fault(domain="EMT", reduce_inertia=True)

### Read dynamic results

In [ ]:
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt

dpsim_result_file = 'logs/' + sim_name_fault_sp + '/' + sim_name_fault_sp + '.csv'
ts_dpsim_sp = read_timeseries_csv(dpsim_result_file)

dpsim_result_file = 'logs/' + sim_name_fault_sp_inertia_reduction + '/' + sim_name_fault_sp_inertia_reduction + '.csv'
ts_dpsim_sp_low_inertia = read_timeseries_csv(dpsim_result_file)

dpsim_result_file = 'logs/' + sim_name_fault_dp + '/' + sim_name_fault_dp + '.csv'
ts_dpsim_dp = read_timeseries_csv(dpsim_result_file)

dpsim_result_file = 'logs/' + sim_name_fault_dp_inertia_reduction + '/' + sim_name_fault_dp_inertia_reduction + '.csv'
ts_dpsim_dp_low_inertia = read_timeseries_csv(dpsim_result_file)

dpsim_result_file = 'logs/' + sim_name_fault_emt + '/' + sim_name_fault_emt + '.csv'
ts_dpsim_emt = read_timeseries_csv(dpsim_result_file)

dpsim_result_file = 'logs/' + sim_name_fault_emt_inertia_reduction + '/' + sim_name_fault_emt_inertia_reduction + '.csv'
ts_dpsim_emt_low_inertia = read_timeseries_csv(dpsim_result_file)

### Load PSAT results

In [ ]:
#if not os.path.exists('reference-results'):
#    os.mkdir('reference-results')

#url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/PSAT/SMIB-Fault/#PSAT_3OrderSyGen_SMIB_Fault_100mS_TS_1mS.out.txt'
#local_file_3Order = 'reference-results/PSAT_3OrderSyGen_SMIB_Fault_100mS_TS_1mS.out'
local_file = './Kundur2Areas/PSAT_Kundur2Areas_Fault_TimeStep_100uS_Tol_1e-5.out'

#urllib.request.urlretrieve(url, local_file_3Order) 
ts_psat = read_timeseries_dpsim(local_file)

#### Power exchange between Area 1 & Area 2

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
from mpl_toolkits.axes_grid1.inset_locator import TransformedBbox, BboxPatch, BboxConnector 

timestep_common = 1e-4
t_begin = 0.0
t_end = 10
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

### Calulate power exchange classical case
power_line3_sp = np.conjugate(ts_dpsim_sp["line3_7-8.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_sp["N7.V"].interpolate(timestep_common).values[begin_idx:end_idx]
power_line4_sp = np.conjugate(ts_dpsim_sp["line4_7-8.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_sp["N7.V"].interpolate(timestep_common).values[begin_idx:end_idx]
power_switch_sp = np.conjugate(ts_dpsim_sp["Switch.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_sp["N8.V"].interpolate(timestep_common).values[begin_idx:end_idx]
#power_switch_sp = 0

power_line3_dp = np.conjugate(ts_dpsim_dp["line3_7-8.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_dp["N7.V"].interpolate(timestep_common).values[begin_idx:end_idx]
power_line4_dp = np.conjugate(ts_dpsim_dp["line4_7-8.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_dp["N7.V"].interpolate(timestep_common).values[begin_idx:end_idx]
power_switch_dp = np.conjugate(ts_dpsim_dp["Switch.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_dp["N8.V"].interpolate(timestep_common).values[begin_idx:end_idx]
#power_switch_sp = 0

power_line3_emt = ts_dpsim_emt["line3_7-8.I_0"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt["N7.V_0"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                  ts_dpsim_emt["line3_7-8.I_1"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt["N7.V_1"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                  ts_dpsim_emt["line3_7-8.I_2"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt["N7.V_2"].interpolate(timestep_common).values[begin_idx:end_idx]
power_line4_emt = ts_dpsim_emt["line4_7-8.I_0"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt["N7.V_0"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                  ts_dpsim_emt["line4_7-8.I_1"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt["N7.V_1"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                  ts_dpsim_emt["line4_7-8.I_2"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt["N7.V_2"].interpolate(timestep_common).values[begin_idx:end_idx]       
power_switch_emt = ts_dpsim_emt["Switch.I_0"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt["N8.V_0"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                   ts_dpsim_emt["Switch.I_1"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt["N8.V_1"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                   ts_dpsim_emt["Switch.I_2"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt["N8.V_2"].interpolate(timestep_common).values[begin_idx:end_idx]
#power_switch_sp = 0

### Calulate power exchange low inertia case
power_line3_sp_low_inertia = np.conjugate(ts_dpsim_sp_low_inertia["line3_7-8.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_dp_low_inertia["N7.V"].interpolate(timestep_common).values[begin_idx:end_idx]
power_line4_sp_low_inertia = np.conjugate(ts_dpsim_sp_low_inertia["line4_7-8.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_dp_low_inertia["N7.V"].interpolate(timestep_common).values[begin_idx:end_idx]
power_switch_sp_low_inertia = np.conjugate(ts_dpsim_sp_low_inertia["Switch.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_dp_low_inertia["N8.V"].interpolate(timestep_common).values[begin_idx:end_idx]

power_line3_dp_low_inertia = np.conjugate(ts_dpsim_dp_low_inertia["line3_7-8.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_dp_low_inertia["N7.V"].interpolate(timestep_common).values[begin_idx:end_idx]
power_line4_dp_low_inertia = np.conjugate(ts_dpsim_dp_low_inertia["line4_7-8.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_dp_low_inertia["N7.V"].interpolate(timestep_common).values[begin_idx:end_idx]
power_switch_dp_low_inertia = np.conjugate(ts_dpsim_dp_low_inertia["Switch.I"].interpolate(timestep_common).values[begin_idx:end_idx]) * ts_dpsim_dp_low_inertia["N8.V"].interpolate(timestep_common).values[begin_idx:end_idx]

power_line3_emt_low_inertia = ts_dpsim_emt_low_inertia["line3_7-8.I_0"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt_low_inertia["N7.V_0"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                              ts_dpsim_emt_low_inertia["line3_7-8.I_1"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt_low_inertia["N7.V_1"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                              ts_dpsim_emt_low_inertia["line3_7-8.I_2"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt_low_inertia["N7.V_2"].interpolate(timestep_common).values[begin_idx:end_idx]
power_line4_emt_low_inertia = ts_dpsim_emt_low_inertia["line4_7-8.I_0"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt_low_inertia["N7.V_0"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                              ts_dpsim_emt_low_inertia["line4_7-8.I_1"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt_low_inertia["N7.V_1"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                              ts_dpsim_emt_low_inertia["line4_7-8.I_2"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt_low_inertia["N7.V_2"].interpolate(timestep_common).values[begin_idx:end_idx]
power_switch_emt_low_inertia = ts_dpsim_emt_low_inertia["Switch.I_0"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt_low_inertia["N8.V_0"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                               ts_dpsim_emt_low_inertia["Switch.I_1"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt_low_inertia["N8.V_1"].interpolate(timestep_common).values[begin_idx:end_idx] + \
                               ts_dpsim_emt_low_inertia["Switch.I_2"].interpolate(timestep_common).values[begin_idx:end_idx] * ts_dpsim_emt_low_inertia["N8.V_2"].interpolate(timestep_common).values[begin_idx:end_idx]    


#fig, ax1 = plt.subplots(figsize=(7, 4))
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 7))
axes[0].plot(time, (power_line3_emt + power_line4_emt - power_switch_emt) * 1e-6, label='EMT')
axes[0].plot(time, (power_line3_dp + power_line4_dp - power_switch_dp).real * 1e-6, '--', label='DP')
axes[0].plot(time, (power_line3_sp + power_line4_sp - power_switch_sp).real  * 1e-6, '--', label='RMS')
axes[0].legend(loc='upper right')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Active power exchange Area 1 - Area 2 (MW)')
axes[0].grid()
axes[0].set_xlim([0.8, 3])
#axes[0].ylim([100, 500])
#plt.ylim([-100, 700])

#second axis of first plot
roi_start_second=0.99
roi_end_second=1.13
ylim = axes[0].get_ylim()
roi = range(int(roi_start_second/timestep_common), int(roi_end_second/timestep_common))
ax2 = axes[0].inset_axes([0.2, 0.15, .7, .55])
#x1, x2, y1, y2 = roi_start_second, roi_end_second, ylim[0], ylim[1]
x1, x2, y1, y2 = roi_start_second, roi_end_second, -300, 750
mark_inset(axes[0], ax2, loc1=2, loc2=3, fc="none", ec='1', color='black')
ax2.plot(time[roi], (power_line3_emt + power_line4_emt - power_switch_emt)[roi] * 1e-6, label='EMT', linewidth=1)   
ax2.plot(time[roi], (power_line3_dp + power_line4_dp - power_switch_dp).real[roi] * 1e-6, '--', label='DP', linewidth=1)   
ax2.plot(time[roi], (power_line3_sp + power_line4_sp - power_switch_sp).real[roi] * 1e-6, '--', label='SP', linewidth=1)    
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax2.set_xlim(x1, x2)
ax2.set_ylim(y1, y2)
ax2.tick_params(top=False, bottom=False, left=False, right=False, labelleft=True, labelbottom=True)
ax2.grid()
plt.tight_layout()
#plt.savefig("./plotting/PowerExchange_ClassicInertia.pdf")

#second plot
axes[1].plot(time, (power_line3_emt_low_inertia + power_line4_emt_low_inertia - power_switch_emt_low_inertia) * 1e-6, label='EMT')
axes[1].plot(time, (power_line3_dp_low_inertia + power_line4_dp_low_inertia - power_switch_dp_low_inertia).real * 1e-6, '--', label='DP')
axes[1].plot(time, (power_line3_sp_low_inertia + power_line4_sp_low_inertia - power_switch_sp_low_inertia).real * 1e-6, '--', label='RMS')
axes[1].legend(loc='lower right')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Active power exchange Area 1 - Area 2 (MW)')
axes[1].grid()
axes[1].set_xlim([0, 10])
#axes[1].set_ylim([-100, 700])
#axes[1].set_ylim([0, 500])

#second axis of second plot
roi_start_second=0.99
roi_end_second=1.13
ylim = axes[1].get_ylim()
roi = range(int(roi_start_second/timestep_common), int(roi_end_second/timestep_common))
ax2 = axes[1].inset_axes([0.18, 0.05, .61, .4])
#x1, x2, y1, y2 = roi_start_second, roi_end_second, ylim[0], ylim[1]
x1, x2, y1, y2 = roi_start_second, roi_end_second, -50, 620
mark_inset(axes[1], ax2, loc1=2, loc2=3, fc="none", ec='1', color='black')
ax2.plot(time[roi], (power_line3_emt_low_inertia + power_line4_emt_low_inertia - power_switch_emt_low_inertia)[roi] * 1e-6, label='EMT')
ax2.plot(time[roi], (power_line3_dp_low_inertia + power_line4_dp_low_inertia - power_switch_dp_low_inertia).real[roi] * 1e-6, '--', label='DP', linewidth=1)   
ax2.plot(time[roi], (power_line3_sp_low_inertia + power_line4_sp_low_inertia - power_switch_sp_low_inertia).real[roi] * 1e-6, '--', label='SP', linewidth=1)    
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax2.set_xlim(x1, x2)
ax2.set_ylim(y1, y2)
ax2.tick_params(top=False, bottom=False, left=False, right=False, labelleft=True, labelbottom=True)
ax2.grid()
plt.tight_layout()
#plt.savefig("./plotting/PowerExchange_LowInertia.pdf")
plt.show()

### Plot results

In [ ]:
timestep_common = 1e-3
t_begin = 0.0
t_end = 10
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 12
height = 4

def plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, 
                       ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, 
                       ts_psat, nominal_voltage, timestep_common=1e-3):
    
    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    dpsim_sp_values_abs_pu = ts_dpsim_sp[varname_dpsim].interpolate(timestep_common).abs().values[begin_idx:end_idx]/nominal_voltage
    dpsim_sp_low_inertia_values_abs_pu = ts_dpsim_sp_low_inertia[varname_dpsim].interpolate(timestep_common).abs().values[begin_idx:end_idx]/nominal_voltage
    dpsim_dp_values_abs_pu = ts_dpsim_dp[varname_dpsim].interpolate(timestep_common).abs().values[begin_idx:end_idx]/nominal_voltage
    dpsim_dp_low_inertia_values_abs_pu = ts_dpsim_dp_low_inertia[varname_dpsim].interpolate(timestep_common).abs().values[begin_idx:end_idx]/nominal_voltage
    dpsim_emt_values_abs_pu = ts_dpsim_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx]/nominal_voltage
    dpsim_emt_low_inertia_values_abs_pu = ts_dpsim_emt_low_inertia[varname_dpsim+"_0"].interpolate(timestep_common).abs().values[begin_idx:end_idx]/nominal_voltage
    psat_values = ts_psat[varname_psat].interpolate(timestep_common).values[begin_idx:end_idx]
    
    plt.figure(figsize=(width, height))
    plt.plot(time, dpsim_sp_values_abs_pu, label='SP - DPsim')
    plt.plot(time, dpsim_dp_values_abs_pu, label='DP - DPsim')
    plt.plot(time, psat_values, '--', label='PSAT')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    #plt.ylim([1.058, 1.062])
    #plt.xlim([0.95, 1.2])
    plt.show()
    
    plt.figure(figsize=(width, height))
    plt.plot(time, dpsim_emt_values_abs_pu * dpsimpy.PEAK1PH_TO_RMS3PH, label='EMT - DPsim')
    plt.plot(time, dpsim_sp_values_abs_pu, '--', label='SP - DPsim')
    plt.plot(time, dpsim_dp_values_abs_pu, '--', label='DP - DPsim')
    
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    plt.xlim([0.95, 1.3])
    plt.show()
    
    """
    plt.figure(figsize=(width, height))
    plt.plot(time, dpsim_dp_low_inertia_values_abs_pu, label='Low Inertia DP - DPsim')
    plt.plot(time, dpsim_sp_low_inertia_values_abs_pu, '--', label='Low Inertia SP - DPsim')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    plt.xlim([0.82, 2])
    plt.show()
    """
    
    #calculate RMSE
    rmse = np.sqrt(((dpsim_sp_values_abs_pu - psat_values) ** 2).mean())
    print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(varname_dpsim, rmse, rmse/1.0*100, 1.0))

#### LV side

In [ ]:
varname_dpsim = 'N1.V'
varname_psat = 'V_Bus01'
nominal_voltage = 20000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage, timestep_common=1e-3)

In [ ]:
varname_dpsim = 'N2.V'
varname_psat = 'V_Bus02'
nominal_voltage = 20000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage)

In [ ]:
varname_dpsim = 'N3.V'
varname_psat = 'V_Bus03'
nominal_voltage = 20000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage)

In [ ]:
varname_dpsim = 'N4.V'
varname_psat = 'V_Bus04'
nominal_voltage = 20000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage)

In [ ]:
varname_dpsim = 'N5.V'
varname_psat = 'V_Bus05'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage)

#### Low voltage side

In [ ]:
varname_dpsim = 'N6.V'
varname_psat = 'V_Bus06'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage)

In [ ]:
varname_dpsim = 'N7.V'
varname_psat = 'V_Bus07'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage)

In [ ]:
varname_dpsim = 'N8.V'
varname_psat = 'V_Bus08'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage)

In [ ]:
varname_dpsim = 'N9.V'
varname_psat = 'V_Bus09'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage)

In [ ]:
varname_dpsim = 'N10.V'
varname_psat = 'V_Bus10'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage)

In [ ]:
varname_dpsim = 'N11.V'
varname_psat = 'V_Bus11'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_dpsim_emt, ts_dpsim_sp_low_inertia, ts_dpsim_dp_low_inertia, ts_dpsim_emt_low_inertia, ts_psat, nominal_voltage)